In [1]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap
import chemiscope

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import spherical

In [2]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# chemiscope.show(frames, mode="structure")

# Equivariance Test - SO(3) for $\nu=3$

In [3]:
# Define target lambda channels
lambdas = np.array([0, 1, 2])

# Pick a test frame
frame = frames[0]

# Randomly rigidly rotate the frame
frame_rot, (a, b, c) = spherical.rotate_ase_frame(frame)
print("Random rotation angles (rad):", a, b, c)
assert not np.all(frame.positions == frame_rot.positions)

# Generate nu=3 descriptor for both frames
nu3 = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)

nu3_rot = clebsch_gordan.n_body_iteration_single_center(
    [frame_rot],
    rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)

# Build a Wigner-D Matrix from the random rotation angles
wig = spherical.WignerDReal(rascal_hypers["max_angular"], a, b, c)

# Rotate the lambda-SOAP descriptor of the unrotated frame
nu3_unrot_rot = wig.rotate_tensormap(nu3)

# Check for equivariance!
assert equistore.equal_metadata(nu3_unrot_rot, nu3_rot)
assert equistore.allclose(nu3_unrot_rot, nu3_rot)
print("SO(3) EQUIVARIANT!")

# chemiscope.show([frames[0], frame_rot], mode="structure")

Random rotation angles (rad): 1.4521811579492145 2.2229088215428923 1.1400953735068622
SO(3) EQUIVARIANT!


# Equivariance Test - O(3) on $\nu=2$ (i.e. $\lambda$-SOAP)

In [4]:
# Define target lambda channels
lambdas = np.array([0, 1, 2, 3, 4, 5])

# Pick a test frame
frame = frames[0]

# Invert the positions
frame_o3 = frame.copy()
frame_o3.positions = -1 * frame_o3.positions
frame_o3, (a, b, c) = spherical.rotate_ase_frame(frame_o3)
print("Random rotation angles (rad):", a, b, c)
assert not np.all(frame.positions == frame_o3.positions)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

lsoap_o3 = clebsch_gordan.n_body_iteration_single_center(
    [frame_o3],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

# Build a Wigner-D Matrix from the random rotation angles
wig = spherical.WignerDReal(rascal_hypers["max_angular"], a, b, c)

# Invert the TensorMap
lsoap_transformed = spherical.invert_tensormap(lsoap)
lsoap_transformed = wig.rotate_tensormap(lsoap_transformed)

# Check for equivariance!
assert equistore.equal_metadata(lsoap_transformed, lsoap_o3)
assert equistore.allclose(lsoap_transformed, lsoap_o3)
print("O(3) EQUIVARIANT!")

# chemiscope.show([frame, frame_o3], mode="structure")

Random rotation angles (rad): 1.5925083910906 0.10309657050925311 1.435700350750785
O(3) EQUIVARIANT!


# Other Tests

In [5]:
# Define target lambda channels
lambdas = np.array([0, 1, 2, 3, 4, 5])

In [6]:
# Dense
lsoap_new0 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=False,
)
lsoap_new0

TensorMap with 33 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         2             1                   0                  3
         2             1                   1                  3
                                    ...
         2             1                   5                  22
         2            -1                   5                  22

In [7]:
# Sparse
lsoap_new1 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
    use_sparse=True,
)
lsoap_new1

TensorMap with 33 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         2             1                   0                  3
         2             1                   1                  3
                                    ...
         2             1                   5                  22
         2            -1                   5                  22

In [8]:
# Check sparse == dense
equistore.allclose(lsoap_new0, lsoap_new1)

True

In [9]:
lsoap_old0 = spherical.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambda_cut=5,
)
# Using the old implementation, blocks must be dropped to retain only the
# desired target lambdas
keys_to_drop = Labels(
    names=lsoap_old0.keys.names,
    values=lsoap_old0.keys.values[[v not in lambdas for v in lsoap_old0.keys.column("spherical_harmonics_l")]],
)
lsoap_old0 = equistore.drop_blocks(lsoap_old0, keys=keys_to_drop)
lsoap_old0

TensorMap with 33 blocks
keys: inversion_sigma  spherical_harmonics_l  species_center
             1                   0                  3
             1                   1                  3
                               ...
            -1                   4                  22
            -1                   5                  22

# Test system 1

In [ ]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body